In [30]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [31]:
df = pd.read_csv('Dataset/Mlbb_Heroes.csv')

In [32]:
df['Hero_Defense'] = df['Mag_Defence'] + df['Phy_Defence']
pd.set_option('display.float_format', '{:.2f}'.format)
df['Win_Rate'] = df['Esport_Wins'] / (df['Esport_Wins'] + df['Esport_Loss'])
df['Hard_Engage'] = (df['Phy_Damage'] + df['Mov_Speed'])
df['Team_Fight'] = (df['Phy_Damage'] + df['Hp'] + df['Hero_Defense'])
df = pd.concat([
    df[df['Primary_Role'] == 'Marksman'], 
    df[df['Primary_Role'] == 'Mage'], 
    df[df['Primary_Role'] == 'Fighter'], 
    df[df['Primary_Role'] == 'Tank'], 
    df[df['Primary_Role'] == 'Support'], 
    df[df['Primary_Role'] == 'Assassin']
    ], axis=0).reset_index(drop=True)
df.index += 1

In [42]:
df

,Name,Title,Voice_Line,Release_Date,Primary_Role,Secondary_Role,Lane,Hp,Hp_Regen,Mana,...,Mag_Damage,Phy_Defence,Mag_Defence,Mov_Speed,Esport_Wins,Esport_Loss,Hero_Defense,Win_Rate,Hard_Engage,Team_Fight
1,Beatrix,Dawnbreak Soldier,"Time and tide wait for no man, not girl.",2021-03-19,Marksman,NaN,Gold Lane,2550,7.20,0,...,0,20,15,257,1016,969,35,0.51,359,2687
2,Brody,The Lone Star,Pain is the proof the I'm still alive.,2020-10-16,Marksman,NaN,Gold Lane,2490,7.20,435,...,0,20,15,253,470,476,35,0.50,358,2630
3,Bruno,Mech Protector,Primary Role,2016,Marksman,NaN,Gold Lane,2522,6.00,439,...,0,17,15,240,154,154,32,0.50,380,2694
4,Claude,Partners in Crime,"Come on Dexter, it's our showtime!",2018-08-07,Marksman,NaN,Gold Lane,2370,8.00,450,...,0,14,15,240,618,583,29,0.51,337,2496
5,Clint,Wild Wanderer,Chivalry is dead. And I will revive it.,2016,Marksman,NaN,Gold Lane,2530,7.20,450,...,0,20,15,240,540,536,35,0.50,355,2680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,Lancelot,Blade of Roses,Thorns remain should petals fall.,2017,Assassin,NaN,Jungler,2549,7.00,450,...,0,16,15,260,603,568,31,0.51,384,2704
111,Ling,Cyan Finch,Still swords run deep.,2019,Assassin,NaN,Jungler,2578,7.80,100,...,0,18,15,260,700,684,33,0.51,385,2736
112,Natalia,Bright Claw,Primary Role,2016-12,Assassin,NaN,Jungler,2639,7.00,486,...,0,18,15,260,173,173,33,0.50,381,2793
113,Selena,Abyssal Witch,Secondary Role,2018-07-18,Assassin,Mage,Jungler,2401,6.80,490,...,0,15,15,240,493,531,30,0.48,350,2541


In [90]:
import numpy as np

def GA_MLBB(dataset, generations, init_pop):
    population = initialize_population(init_pop, 5)
    fit_value = []
    for individual in population:
        a= fitness_starg1(dataset, individual)
        fit_value.append(a)
    fit_value

    for i in range(1,generations):
        #select parent
        parents= selection(4, population, fit_value)
        child=crossover_4_parents(parents[0], parents[1], parents[2], parents[3])

        #add child ke population
        population.extend(child)
        wr_value = []
        for individual in population:
            a= func_mutation(individual)
            wr_value.append(a)
        
        #hasil mutasi
        mutation_result= [value for value in wr_value if value > 0.50]
        if len(mutation_result)<4:
            wr_value= wr_value.sort(reverse=True)
            mutation_result = wr_value[1:4]

        new_pop= []
        index =[]
        for i in mutation_result:
            ind = wr_value.index(i)
            index.append(ind)
        
        for i in index:
            new_pop.append(population[i])

        return new_pop

#fitness Function
def fitness_starg1(df,team):
    ind1=0
    ind2=0
    for h in team:
        ind1 += df['Hard_Engage'][h]
        ind2 += df['Win_Rate'][h]
    fit = ind1 * (ind2/5)
    return fit

#Init population
import random
def generate_individual(chromosome_length):
    def random_integer_except(excluded_value):
        while True:
            random_value = random.randint(44, 94)  # Ganti rentang sesuai kebutuhan Anda
            if random_value != excluded_value:
                return random_value
    # Fungsi ini menghasilkan satu individu acak dengan panjang kromosom tertentu.
    individual= [0]*chromosome_length
    individual[0] = random.randint(1,18)
    individual[1] = random.randint(19,43)
    individual[2] = random.randint(44,76)
    individual[3] = random.randint(77,101)
    individual[4] = random.choice([random_integer_except([individual[2],individual[3]]), random.randint(102,114)])

    return individual

def initialize_population(population_size, chromosome_length):
    # Fungsi ini menghasilkan populasi awal dengan sejumlah individu.
    return [generate_individual(chromosome_length) for _ in range(population_size)]

#selection
import heapq
def selection(num_select, population, fit_value):
    fit_selected = heapq.nlargest(num_select,fit_value)
    parents1= population[fit_value.index(fit_selected[0])]
    parents2= population[fit_value.index(fit_selected[1])]
    parents3= population[fit_value.index(fit_selected[2])]
    parents4= population[fit_value.index(fit_selected[3])]
    parents= [parents1, parents2, parents3, parents4]
    return parents

#cross-over
def crossover_4_parents(parent1, parent2, parent3, parent4):
    # Menghitung panjang kromosom
    chromosome_length = len(parent1)

    # Memilih dua titik crossover acak
    crossover_point1 = random.randint(1, chromosome_length - 1)
    crossover_point2 = random.randint(1, chromosome_length - 1)

    # Memastikan titik crossover1 dan crossover2 berbeda
    while crossover_point1 == crossover_point2:
        crossover_point2 = random.randint(1, chromosome_length - 1)

    # Mengurutkan titik crossover
    crossover_point1, crossover_point2 = min(crossover_point1, crossover_point2), max(crossover_point1, crossover_point2)

    # Membuat empat anak baru dengan menggabungkan gen dari empat orangtua
    child1 = parent1[:crossover_point1] + parent2[crossover_point1:crossover_point2] + parent3[crossover_point2:] + parent4[crossover_point2:]
    child2 = parent3[:crossover_point1] + parent4[crossover_point1:crossover_point2] + parent1[crossover_point2:] + parent2[crossover_point2:]
    child3 = parent2[:crossover_point1] + parent1[crossover_point1:crossover_point2] + parent4[crossover_point2:] + parent3[crossover_point2:]
    child4 = parent4[:crossover_point1] + parent3[crossover_point1:crossover_point2] + parent2[crossover_point2:] + parent1[crossover_point2:]

    child=[child1, child2, child3, child4]
    return child

#mutation_function
def func_mutation(team):
    wr=0
    for h in team:
        wr += df['Win_Rate'][h]

    wr = wr/5
    return wr
#mutation_percent_gens


In [101]:
parents= GA_MLBB(df, 1000, 30)
parents

[[2, 27, 75, 100, 107],
 [7, 41, 49, 99, 74],
 [14, 37, 52, 86, 92],
 [4, 30, 70, 83, 110],
 [1, 32, 71, 90, 73],
 [5, 29, 66, 89, 59],
 [14, 39, 69, 81, 67],
 [5, 19, 70, 100, 67],
 [10, 36, 67, 89, 112],
 [5, 41, 70, 83, 110, 52, 86, 92],
 [4, 37, 66, 89, 59, 49, 99, 74],
 [7, 29, 52, 86, 92, 70, 83, 110],
 [14, 30, 49, 99, 74, 66, 89, 59]]

In [102]:
fit_value = []
for individual in parents:
    a= fitness_starg1(df, individual)
    fit_value.append(a)

fit_value

[926.763600108394,
 961.101084777609,
 947.0141912737479,
 957.1814805459328,
 939.0223709265358,
 961.3317681252148,
 907.556091318138,
 941.1182840253284,
 930.0009384570718,
 2484.8880605195563,
 2429.3355946389343,
 2519.524447125015,
 2459.9629899602182]